# Loading data

In [51]:
import cryptocompare
import datetime
import pandas as pd
cc=cryptocompare.get_historical_price_hour('BAT', curr='USDT',limit=10, toTs=datetime.datetime.now())
pdbinanceframe=pd.DataFrame(cc)

# Check data time

In [53]:
import time
time.ctime(int(pdbinanceframe[-1:]['time']))

'Mon Jun  8 17:00:00 2020'

# Trading

In [ ]:
import ccxt
import binance
import pandas as pd
import datetime, time
from decimal import Decimal


from binance.client import Client
clientsammy = Client('xxxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxxxxxxx')

binanceapi = ccxt.binance({
    'apiKey': 'xxxxxxxxxxxxxxxxxxxxxxxxxxx',
    'secret': 'xxxxxxxxxxxxxxxxxxxxxxxxxxx',
    'enableRateLimit': True,
})


executed=0
closed=0


class dataandtrade:
    
  def __init__(self, exchange, symbol, timeframe, month, day, hour, minute, historyhighlastrows, historylowlastrows, ordersize, sl, tp, price, direction, maxdeviation, maxspread, pdohlcv, digits, lowestprop, highestprop, clientsammy, orderpercentage):
        
        
        self.exchange = exchange
        self.symbol = symbol
        self.timeframe = timeframe
        self.month = month
        self.day = day
        self.hour = hour
        self.minute = minute
        self.historyhighlastrows = historyhighlastrows
        self.historylowlastrows = historylowlastrows
        self.ordersize = ordersize
        self.sl = sl
        self.tp = tp
        self.price = price
        self.direction = direction
        self.maxdeviation = maxdeviation
        self.maxspread = maxspread
        self.pdohlcv=pdohlcv
        self.orderbook=self.get_orderbook()
        self.ask=self.get_ask()
        self.bid=self.get_bid()
        self.digits=digits
        self.lowestprop=lowestprop
        self.highestprop=highestprop
        self.clientsammy=clientsammy
        self.orderpercentage=orderpercentage
        #self.balance_futures=self.get_futures_balance()
        
  def futuresactivation(self):
    #CCXT futures trading untested
    exchange.options = {'defaultType': 'future'}
    markets = exchange.load_markets()
  
  def get_dataframe_ccxt(self):

    ohlcv = self.exchange.fetch_ohlcv(self.symbol, self.timeframe)
    pdohlcv = pd.DataFrame(ohlcv)
    return pdohlcv
    #pdohlcv.columns = ['time','open','high','low','close','volume']
  
  def get_dataframe(self):
    symbolnew=self.symbol.replace('/','')
    ohlcv = binance.klines(symbolnew, self.timeframe)
    pdohlcv = pd.DataFrame(ohlcv)
    return pdohlcv

  def get_timepoint(self):
    t = datetime.datetime(2020, self.month, self.day, self.hour, self.minute)
    tradedate=time.mktime(t.timetuple())
    diffday=self.pdohlcv['time'][2]-self.pdohlcv['time'][1]
    tradedate4use=tradedate-diffday
    return tradedate4use
    
  def get_pdohlcvnew(self):
    tradedate4use=self.get_timepoint()
    pdohlcvnew=self.pdohlcv[self.pdohlcv['time']>(tradedate4use)]
    return pdohlcvnew
  
  def get_pdohlcvhistory(self):
    tradedate4use=self.get_timepoint()
    pdohlcvhistory=self.pdohlcv[self.pdohlcv['time']<=(tradedate4use)]
    return pdohlcvhistory
    
    #Low and High after Tradedate 0=time 1=open 2=high 3=low 4=close 5=volume
  def get_highest(self):
    pdohlcvnew=self.get_pdohlcvnew()
    lasthighnumbers=pdohlcvnew['high']
    highest=max(lasthighnumbers)
    return float(highest)
    
  def get_lowest(self):
    pdohlcvnew=self.get_pdohlcvnew()
    lastlownumbers=pdohlcvnew['low']
    lowestold=min(lastlownumbers)
    
    return float(lowestold)
    
    #Low and High before Tradedate
  def get_maxhistory(self):
    pdohlcvhistory = self.get_pdohlcvhistory()
    maxhistory=max(pdohlcvhistory[-self.historyhighlastrows:]['high'])
    return float(maxhistory)

  def get_minhistory(self):
    pdohlcvhistory = self.get_pdohlcvhistory()
    minhistory=min(pdohlcvhistory[-self.historylowlastrows:]['low'])
    return float(minhistory)

  def get_orderbook(self):
    try:
        orderbook = self.exchange.fetch_order_book(self.symbol,5)
    except ccxt.ExchangeNotAvailable as e:
        print('No data')
        print('e')
    return orderbook

  def get_bid(self):
    orderbook=self.orderbook
    bid = orderbook['bids'][0][0] if len (orderbook['bids']) > 0 else None
    return bid

  def get_ask(self):
    orderbook=self.orderbook
    ask = orderbook['asks'][0][0] if len (orderbook['asks']) > 0 else None
    return ask

  def entryprice_long(self):
      entryprice=(float(self.get_maxhistory())*self.price/100) + float(self.get_maxhistory())
      return float(entryprice)
      #The Bid must be above this price
    
  def marketorder_long(self):
      extras=''
      order = self.exchange.create_order(self.symbol, 'market', 'buy', self.ordersize, self.ask, extras)
      #direction buy or sell
        
  def marketorder_short(self):
      extras=''
      order = self.exchange.create_order(self.symbol, 'market', 'sell', self.ordersize, self.bid, extras)
    
  def ocoorder_long(self):
      symbolnew=self.symbol.replace('/','')
      order = self.exchange.private_post_order_oco({ "symbol": symbolnew, "side": "sell", "quantity": self.ordersize, "price": self.takeprofit_long(), "stopPrice": self.stoploss_long(), "stopLimitPrice": self.stoplosslimit_long(), "stopLimitTimeInForce": "GTC"})
      #Use symbol without / symbol=self.symbol.replace('/','')
    
  def ocoordertest(self):
      order = binance.private_post_order_oco({ "symbol": "ETHBTC", "side": "buy", "quantity": 0.005, "price": 0.02489393994, "stopPrice": 0.025848484949, "stopLimitPrice": 0.03838488484, "stopLimitTimeInForce": "GTC"})
      return order
  
  def futuresorder_long(self):
    #python binance margin order is in docs as well  
    symbolnew=self.symbol.replace('/','')
    order = self.clientsammy.futures_create_order(symbol=symbolnew, side='BUY', type='MARKET', quantity=self.ordersize)
    return order

  def futuresorder_short(self):
    #python binance margin order is in docs as well  
    symbolnew=self.symbol.replace('/','')
    order = self.clientsammy.futures_create_order(symbol=symbolnew, side='SELL', type='MARKET', quantity=self.ordersize)
    return order
    
  def limitordertest_long(self):
      extras=''
      order = self.exchange.create_order("ETH/BTC", 'limit', 'buy', 0.007, 0.01500194884, extras)
      return order
      #direction buy or sell
    
  def stoploss_long(self):
      ask=self.ask
      bid=self.bid
      shortenstring='{0:.' + str(self.digits) + 'f}'
      stoplossprice=((self.get_minhistory()-(self.get_minhistory()*self.sl/100))-(ask-bid))
      stoplossprice=round(float(stoplossprice),self.digits)

      return stoplossprice
    
  def stoplosslimit_long(self):
      shortenstring='{0:.' + str(self.digits) + 'f}'
      stoplosslimit=self.bid*0.7
      stoplosslimit=round(stoplosslimit,self.digits)
      return float(stoplosslimit)

  def get_marketstructure(self):
      self.exchange.load_markets()
      structure=self.exchange.market(self.symbol)
      return structure
  
  def get_digits(self):
      digits=self.get_marketstructure()['precision']['price']
      return digits  
      
  def takeprofit_long_old(self):
      shortenstring='{0:.' + str(self.digits) + 'f}'
      takeprofit_price=self.get_maxhistory()+(self.get_maxhistory()*self.tp/100)+(self.ask-self.bid)
      takeprofit_price=round(takeprofit_price, self.digits)
      #takeprofit_price=shortenstring.format(takeprofit_price)
      return float(takeprofit_price)
        
  def deviation(self):
      if self.lowestprop < self.get_lowest():
            lowest2use=self.lowestprop
      else: 
            lowest2use=self.get_lowest()
      
      deviation=(lowest2use-self.get_minhistory())/(self.get_maxhistory()-self.get_minhistory())*100
      return float(deviation)
  
  def get_last_order_futures(self):
      symbolnew=self.symbol.replace('/','')
      theorder=self.clientsammy.futures_get_all_orders(symbol=symbolnew,limit=1)
      return theorder
  
  def get_execution_quantity_futures(self):
      quantity=self.get_last_order_futures()[0]['executedQty']
      return quantity

  def get_futures_balance(self):
      amount=self.clientsammy.futures_account_balance()[0]['balance']
      return float(amount)

  def get_ordersize_futures(self):
      theordersize=self.get_futures_balance()*self.orderpercentage/100/((self.get_maxhistory()-self.get_minhistory())/self.get_maxhistory())/self.bid
      if   theordersize >= 10:
             theordersize=round(theordersize,0)

      elif theordersize >=1:
             theordersize=round(theordersize,1)
             
      elif theordersize >=0.1:
             theordersize=round(theordersize,2)
             
      elif theordersize >=0.01:
             theordersize=round(theordersize,3)
             
      elif theordersize >=0.001:
             theordersize=round(theordersize,4)
            
      else:
           theordersize=round(theordersize,5)
            
      return theordersize
    
  def get_takeprofit(self):
      newtp=(self.get_maxhistory()-self.get_minhistory())*self.tp+self.get_maxhistory()*(1+self.price/100)
      return newtp
        
  def checklongentry(self):
      spreadp=((self.ask-self.bid)/self.bid*100)
      if self.bid>self.entryprice_long() and spreadp<self.maxspread and self.deviation() > self.maxdeviation and self.bid<(self.entryprice_long()*1.5) and self.highestprop<(self.entryprice_long()*1.5) and self.get_highest()<(self.entryprice_long()*1.5):
        
         execute=1
            
      else: 
         
         execute=0
        
      return execute

thesymbol='BAT/USDT'
month=6
day=8
hour=17
minute=23
historyhighlastrows=7 #Lookbacknumber
historylowlastrows=2
orderpercentage=30 #percent of capital
size=10 #Initial only
sl=0.1 #Percentage from low
tp=3 #Multiple of historical high and low difference
price=0.3 #Percentage from high
mdev=-40 #Max deviation of current range
maxspread=2 #Bid and Ask Spread
digits=4 #just initially
lowest=1000000.7 #just initially
highest=0.0000000000000001 #just initially


                        #exchange symbol    timeframe  month  day  hour  minute  historyhighlastrows  historylowlastrows  size  sl  tp  price side mdev maxspread  
databinance = dataandtrade(binanceapi, thesymbol, thetf, month, day,hour, minute,     historyhighlastrows, historylowlastrows,  size, sl,tp,price, 'buy',mdev,   maxspread, pdbinanceframe, digits, lowest, highest, clientsammy, orderpercentage)

digits=databinance.get_digits()
size=databinance.get_ordersize_futures()

printmh=databinance.get_maxhistory()
print(printmh)
printml=databinance.get_minhistory()
print(printml)
printh=databinance.get_highest()
print(printh)
printl=databinance.get_lowest()
print(printl)
printep=databinance.entryprice_long()
print(printep)
printsl=databinance.stoploss_long()
print(printsl)
printsll=databinance.stoplosslimit_long()
print(printsll)
printtp=databinance.get_takeprofit()
print(printtp)
printdev=databinance.deviation()
print(printdev)
printtheos=databinance.get_ordersize_futures()
print(printtheos)


while True:
  databinance = dataandtrade(binanceapi, thesymbol, thetf, month, day,hour, minute,     historyhighlastrows, historylowlastrows,  size, sl,tp,price, 'buy',mdev,   maxspread, pdbinanceframe, digits, lowest, highest, clientsammy, orderpercentage)
  if databinance.bid<lowest:
        lowest=databinance.bid
  if databinance.bid>highest:
        highest=databinance.bid
  if databinance.checklongentry()==1 and executed==0:
        databinance.futuresorder_long()
        executed=1
        time.sleep(5)
        size=databinance.get_execution_quantity_futures()
  if executed ==1 and databinance.bid < databinance.stoploss_long() and closed==0:
        databinance.futuresorder_short()
        closed=1
  if executed ==1 and databinance.bid > databinance.get_takeprofit() and closed==0:
        databinance.futuresorder_short()
        closed=1

0.2541
0.2467
0.2489
0.2465
0.2548623
0.2462
0.1727
0.2770623
-2.702702702702784
70.0


Add the API keys two times for both packages. If a trade is open or closed change the parameters accordingly.